# Decision Tree Implementation Assignment
### By:  John Pazzelli
(Part of the 4 course Machine Learning specialization on Coursera)  
**Nov. 2017**


## Assignment Background
- The assignment **dataset contains loan information** for a company that connects those looking for a loan with those who are willing to lend money to these loanees for a pre-specified rate of return
- The **task of this assignment** was to **hand-code a decision tree** and to **use it to classify bad loans from good loans** based on the terms of the loan and characteristics of the loanee

## Decision Tree Implementation
- Although we were provided starter code, I did not use it (primarily because it was not vectorized) and instead **created my own object-oriented implementation of a decision tree** with early stopping conditions
- My manual implementation is contained in the **DecisionTree** and **TreeNode classes** below

In [1]:
# Libraries
import pandas as pd
import numpy as np
from IPython.display import display
pd.set_option('display.max_columns', None)

In [114]:
# Read loan dataset and create target 'safe_loans' column
df = pd.read_csv('..\Wk3\lending-club-data.csv')
df['safe_loans'] = df['bad_loans'].apply(lambda val: 1 if val == 0 else 0)
del df['bad_loans']
display(df.head())
print(df.columns)

c:\program files\python36\lib\site-packages\IPython\core\interactiveshell.py:2728: DtypeWarning: Columns (19,47) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


,id,member_id,loan_amnt,funded_amnt,funded_amnt_inv,term,int_rate,installment,grade,sub_grade,emp_title,emp_length,home_ownership,annual_inc,is_inc_v,issue_d,loan_status,pymnt_plan,url,desc,purpose,title,zip_code,addr_state,dti,delinq_2yrs,earliest_cr_line,inq_last_6mths,mths_since_last_delinq,mths_since_last_record,open_acc,pub_rec,revol_bal,revol_util,total_acc,initial_list_status,out_prncp,out_prncp_inv,total_pymnt,total_pymnt_inv,total_rec_prncp,total_rec_int,total_rec_late_fee,recoveries,collection_recovery_fee,last_pymnt_d,last_pymnt_amnt,next_pymnt_d,last_credit_pull_d,collections_12_mths_ex_med,mths_since_last_major_derog,policy_code,not_compliant,status,inactive_loans,emp_length_num,grade_num,sub_grade_num,delinq_2yrs_zero,pub_rec_zero,collections_12_mths_zero,short_emp,payment_inc_ratio,final_d,last_delinq_none,last_record_none,last_major_derog_none,safe_loans
0,1077501,1296599,5000,5000,4975,36 months,10.65,162.87,B,B2,NaN,10+ years,RENT,24000.0,Verified,20111201T000000,Fully Paid,n,https://www.lendingclub.com/browse/loanDetail....,Borrower added on 12/22/11 > I need to upgra...,credit_card,Computer,860xx,AZ,27.65,0.0,19850101T000000,1.0,NaN,NaN,3.0,0.0,13648,83.7,9.0,f,0.0,0.0,5861.07,5831.78,5000.00,861.07,0.00,0.00,0.00,20150101T000000,171.62,NaN,20150101T000000,0.0,NaN,1,0,Fully Paid,1,11,5,0.4,1.0,1.0,1.0,0,8.14350,20141201T000000,1,1,1,1
1,1077430,1314167,2500,2500,2500,60 months,15.27,59.83,C,C4,Ryder,< 1 year,RENT,30000.0,Source Verified,20111201T000000,Charged Off,n,https://www.lendingclub.com/browse/loanDetail....,Borrower added on 12/22/11 > I plan to use t...,car,bike,309xx,GA,1.00,0.0,19990401T000000,5.0,NaN,NaN,3.0,0.0,1687,9.4,4.0,f,0.0,0.0,1008.71,1008.71,456.46,435.17,0.00,117.08,1.11,20130401T000000,119.66,NaN,20130901T000000,0.0,NaN,1,0,Charged Off,1,1,4,0.8,1.0,1.0,1.0,1,2.39320,20161201T000000,1,1,1,0
2,1077175,1313524,2400,2400,2400,36 months,15.96,84.33,C,C5,NaN,10+ years,RENT,12252.0,Not Verified,20111201T000000,Fully Paid,n,https://www.lendingclub.com/browse/loanDetail....,NaN,small_business,real estate business,606xx,IL,8.72,0.0,20011101T000000,2.0,NaN,NaN,2.0,0.0,2956,98.5,10.0,f,0.0,0.0,3003.65,3003.65,2400.00,603.65,0.00,0.00,0.00,20140601T000000,649.91,NaN,20150201T000000,0.0,NaN,1,0,Fully Paid,1,11,4,1.0,1.0,1.0,1.0,0,8.25955,20141201T000000,1,1,1,1
3,1076863,1277178,10000,10000,10000,36 months,13.49,339.31,C,C1,AIR RESOURCES BOARD,10+ years,RENT,49200.0,Source Verified,20111201T000000,Fully Paid,n,https://www.lendingclub.com/browse/loanDetail....,Borrower added on 12/21/11 > to pay for prop...,other,personel,917xx,CA,20.00,0.0,19960201T000000,1.0,35.0,NaN,10.0,0.0,5598,21.0,37.0,f,0.0,0.0,12226.30,12226.30,10000.00,2209.33,16.97,0.00,0.00,20150101T000000,357.48,NaN,20150101T000000,0.0,NaN,1,0,Fully Paid,1,11,4,0.2,1.0,1.0,1.0,0,8.27585,20141201T000000,0,1,1,1
4,1075269,1311441,5000,5000,5000,36 months,7.90,156.46,A,A4,Veolia Transportaton,3 years,RENT,36000.0,Source Verified,20111201T000000,Fully Paid,n,https://www.lendingclub.com/browse/loanDetail....,NaN,wedding,My wedding loan I promise to pay back,852xx,AZ,11.20,0.0,20041101T000000,3.0,NaN,NaN,9.0,0.0,7963,28.3,12.0,f,0.0,0.0,5631.38,5631.38,5000.00,631.38,0.00,0.00,0.00,20150101T000000,161.03,NaN,20150201T000000,0.0,NaN,1,0,Fully Paid,1,4,6,0.8,1.0,1.0,1.0,0,5.21533,20141201T000000,1,1,1,1


Index(['id', 'member_id', 'loan_amnt', 'funded_amnt', 'funded_amnt_inv',
       'term', 'int_rate', 'installment', 'grade', 'sub_grade', 'emp_title',
       'emp_length', 'home_ownership', 'annual_inc', 'is_inc_v', 'issue_d',
       'loan_status', 'pymnt_plan', 'url', 'desc', 'purpose', 'title',
       'zip_code', 'addr_state', 'dti', 'delinq_2yrs', 'earliest_cr_line',
       'inq_last_6mths', 'mths_since_last_delinq', 'mths_since_last_record',
       'open_acc', 'pub_rec', 'revol_bal', 'revol_util', 'total_acc',
       'initial_list_status', 'out_prncp', 'out_prncp_inv', 'total_pymnt',
       'total_pymnt_inv', 'total_rec_prncp', 'total_rec_int',
       'total_rec_late_fee', 'recoveries', 'collection_recovery_fee',
       'last_pymnt_d', 'last_pymnt_amnt', 'next_pymnt_d', 'last_credit_pull_d',
       'collections_12_mths_ex_med', 'mths_since_last_major_derog',
       'policy_code', 'not_compliant', 'status', 'inactive_loans',
       'emp_length_num', 'grade_num', 'sub_grade_num', 'del

In [115]:
# Reduce down to features that will be used in decision tree and define target (output) column
features = ['grade', 'term', 'home_ownership', 'emp_length']
target = 'safe_loans'
df = df[features + [target]]

In [116]:
# Read indices of training and validation set
# An external file was used so all students would use identical indices for reproducible results
train_idx = pd.read_json('module-6-assignment-train-idx.json').loc[:,0].values
valid_idx = pd.read_json('module-6-assignment-validation-idx.json').loc[:,0].values

In [117]:
# One-hot encoding and reorder columns to place target column at the end
df_dum = pd.get_dummies(df)
cols = df_dum.columns.tolist()
cols.remove(target)
cols.append(target)
df_dum = df_dum[cols]

# Split into train / test sets
df_train = df_dum.iloc[train_idx]
df_valid = df_dum.iloc[valid_idx]

In [118]:
# Output number of positive and negative training examples
# This dataset has balanced classes
print(df_train[target].value_counts())
print(df_train[target].unique())

1    18748
0    18476
Name: safe_loans, dtype: int64
[0 1]


## Manual Decision Tree Implementation
### TreeNode Class
- **Represents a single node** of a decision tree
- Tracks the **predicted label for that node** as well as the **number of classification mistakes** it makes (i.e. the number of examples assigned to this node whose target / output value does not fall into the majority class)
- This class **also contains code to determine the best feature to use to split this node** assuming that this split would provide the necessary **minimum error reduction** and would not violate the **minimum node size** constraint

In [119]:
class TreeNode:
    def __init__(self, df, target, depth, min_node_size, min_error_reduction):
        self.df = df   # Input DataFrame
        self.target = target   # Target feature name (the one used to classify this example)
        self.depth = depth  # Depth of this node in the tree, needed to prevent splitting if this node is at max_depth
        self.min_node_size = min_node_size  # Minimum node size below which this node will not be further split
        self.min_error_reduction = min_error_reduction  # Min. error reduction required to split this node
    
    def is_leaf(self): return not hasattr(self, 'children')
    def print_info(self): print("\nNode Depth: {0}\nIs leaf? {1}\nSplits on: '{2}'\nClassification Error: {3}\nTarget value counts: {4}".format(self.depth, self.is_leaf(), (self.best_splitting_feature() if not self.is_leaf() else 'N/A'), self.prct_mistakes(), self.target_value_counts().values))
    
    def target_value_counts(self): 
        if not hasattr(self, 'target_values'): self.target_values = self.df[self.target].value_counts()
        return self.target_values
    
    def predict_label(self): 
        if not hasattr(self, 'predicted_label'): self.predicted_label = self.target_value_counts().idxmax()
        return self.predicted_label
    
    # Number of classification mistakes this node makes
    def num_mistakes(self): 
        if not hasattr(self, 'mistake_count'): self.mistake_count = len(self.df[self.df[self.target] != self.predict_label()])
        return self.mistake_count
    
    def prct_mistakes(self):
        if not hasattr(self, 'mistake_prct'): self.mistake_prct = self.num_mistakes() / float(len(self.df))
        return self.mistake_prct
    
    # Determines the error that would result by splitting on the input feature
    # It does this by creating a new, temporary TreeNode() for each value of the target feature and summing the number of mistakes
    def calc_feature_split_error(self, feature):
        return np.sum([TreeNode(self.df[self.df[feature] == value], self.target, self.depth+1, self.min_node_size, self.min_error_reduction).num_mistakes() for value in self.df[feature].unique()]) / float(len(self.df))
    
    def best_split_feature_values(self):
        if not hasattr(self, 'splitting_feature_values'): self.splitting_feature_values = sorted(self.df[self.best_splitting_feature()].unique())
        return self.splitting_feature_values
    
    def best_split_feature_error(self):
        if not hasattr(self, 'splitting_feature_error'): self.best_splitting_feature()
        return self.splitting_feature_error
    
    # Determine the best feature on which to split this node
    # It does this by calculating the error if splitting on each feature and taking the lowest of the errors
    def best_splitting_feature(self):
        if not hasattr(self, 'splitting_feature'):
            features = [f for f in self.df.columns.tolist() if f != self.target]
            errors = [self.calc_feature_split_error(feature) for feature in features]
            best_idx = np.argmin(errors)
            self.splitting_feature = features[best_idx]
            self.splitting_feature_error = errors[best_idx]
        return self.splitting_feature
    
    # Determines if the splitting conditions have been met and if so, splits the node on the best feature (the one that 
    # provides best classification error reduction) by partitioning the input DataTable on each separate value for that feature
    def try_split_node(self, max_depth):
        # Don't split if: no more features to split / node is at max_depth / node has fewer than min_samples / no misclassified samples at this node
        if len(self.df.columns.tolist()) - 1 <= 0 or self.depth >= max_depth or len(self.df) <= self.min_node_size or (self.prct_mistakes() - self.best_split_feature_error()) <= self.min_error_reduction or self.num_mistakes() <= 0: 
            return False
        
        # Create one child per value of the best feature and create an alias for left and right nodes (only relevant for binary trees)
        new_features = self.df.columns[self.df.columns != self.best_splitting_feature()].tolist()
        self.children = [TreeNode(self.df[self.df[self.best_splitting_feature()] == val].loc[:, new_features], self.target, self.depth + 1, self.min_node_size, self.min_error_reduction) for val in self.best_split_feature_values()]
        if len(self.children) == 2: 
            self.left = self.children[0]
            self.right = self.children[1]
        return True

### DecisionTree Class
- Represents a **decision tree** whose root node is tracked via the self.root pointer
- Supports some **parameters** that control the **minimum node size** required to split a node, the **maximum depth** of the tree and the **minimum error reduction** required to split a node during training

In [161]:
class DecisionTree:
    def __init__(self, df, target, min_node_size=1, max_depth=10, min_error_reduction=0.0):
        self.df = df  # Input DataFrame
        self.target = target  # Target feature name (the feature that is used to classify each example)
        self.max_depth = max_depth  # Maximum depth of the tree
        self.min_node_size = min_node_size  # Minimum size of the node required for it to be split
        self.min_error_reduction = min_error_reduction  # Minimum error reduction required to split a node
        self.leaf_nodes = []
        self.root = TreeNode(self.df, self.target, depth=1, min_node_size=min_node_size, min_error_reduction=min_error_reduction)
        self.build_tree(self.root)  # Kick-off the recursive tree-building procedure
    
    # Recursive function that splits the input node and its children until no more splits can be made
    def build_tree(self, node):
        if node.try_split_node(self.max_depth): list(map(self.build_tree, node.children))
        else: self.leaf_nodes.append(node)
        
    def count_leaves(self): return len(self.leaf_nodes)
    
    # Recursive function to classify the input rows in the DataFrame df
    # This is done by calling node.predict_label() if the node is a leaf node and applying the prediction to all rows
    # or if not a leaf node, the input DataFrame is partitioned the same way that the current node is split
    # and this function is called again with each sub-table on its appropriate child node
    def classify_recursive(self, node, df, annotate=False):
        # If a leaf node, use the node's predicted label for all the input examples
        if node.is_leaf(): 
            if annotate: print("At leaf, {0} rows predicted {1}".format(len(df), node.predict_label()))
            return pd.Series(np.repeat(node.predict_label(), len(df)), index=df.index)
        
        # If not a leaf node, split the DataFrame df into sub-tables as per the node's feature split
        # and call this function recursively on each sub-table with the appropriate child node
        child_series = []
        for i,child in enumerate(node.children):
            child_df = df[df[node.best_splitting_feature()] == node.best_split_feature_values()[i]]
            if len(child_df) <= 0: continue
            if annotate: print("Split {0} rows on {1} = {2}".format(len(child_df), node.best_splitting_feature(), node.best_split_feature_values()[i]))
            child_series.append(self.classify_recursive(child, child_df, annotate))
        return pd.concat(child_series)
    
    # Classify the rows of the input DataFrame, X_df using recursive classification logic
    def classify(self, X_df, annotate=False):
        return self.classify_recursive(self.root, pd.DataFrame(X_df), annotate)
    
    # Accuracy score for classification of the input X_test DataFrame against the y_test labels
    # Use the XOR operation between the predicted and actual labels to identify errors
    def score(self, X_test, y_test):
        y_pred = self.classify(X_test).sort_index().as_matrix()
        if len(y_pred) != len(y_test): y_pred = np.pad(y_pred, (0, len(y_test) - len(y_pred)), mode='constant')
        return 1 - (np.sum(y_pred ^ y_test.sort_index().as_matrix()) / float(len(y_test)))
    
    def print_info(self):
        self.print_node_info(self.root)
    
    def print_node_info(self, node):
        node.print_info()
        if hasattr(node, 'children'): list(map(lambda child: self.print_node_info(child), node.children))

## Note: 
The content below contains the questions asked in the assignment and answers them using my manual decision tree implementation above.  **Feel free to skip this section if necessary.**

# Quiz question
Given an intermediate node with 6 safe loans and 3 risky loans, if the min_node_size parameter is 10, what should the tree learning algorithm do next?

In [154]:
print("Don't split - move on to another node")

Don't split - move on to another node


# Quiz question
Assume an intermediate node has 6 safe loans and 3 risky loans. For each of 4 possible features to split on, the error reduction is 0.0, 0.05, 0.1, and 0.14, respectively. If the minimum gain in error reduction parameter is set to 0.2, what should the tree learning algorithm do next?

In [122]:
print("Don't split - move on to another node")

Don't split - move on to another node


In [123]:
#%%timeit -r 3 -n 3
dt_new = DecisionTree(df_train, target, max_depth=6, min_node_size=100, min_error_reduction=0.0)

In [124]:
dt_old = DecisionTree(df_train, target, max_depth=6, min_node_size=0, min_error_reduction=-1)

In [127]:
dt_new.print_info()


Node Depth: 1
Is leaf? False
Splits on: 'term_ 36 months'
Classification Error: 0.4963464431549538
Target value counts: [18748 18476]

Node Depth: 2
Is leaf? False
Splits on: 'grade_A'
Classification Error: 0.34923560663558495
Target value counts: [6002 3221]

Node Depth: 3
Is leaf? True
Splits on: 'N/A'
Classification Error: 0.34630563472922604
Target value counts: [5963 3159]

Node Depth: 3
Is leaf? False
Splits on: 'emp_length_< 1 year'
Classification Error: 0.38613861386138615
Target value counts: [62 39]

Node Depth: 4
Is leaf? True
Splits on: 'N/A'
Classification Error: 0.36666666666666664
Target value counts: [57 33]

Node Depth: 4
Is leaf? True
Splits on: 'N/A'
Classification Error: 0.45454545454545453
Target value counts: [6 5]

Node Depth: 2
Is leaf? False
Splits on: 'grade_D'
Classification Error: 0.4454840898539338
Target value counts: [15527 12474]

Node Depth: 3
Is leaf? False
Splits on: 'grade_E'
Classification Error: 0.417725321888412
Target value counts: [13567  9733]

In [128]:
dt_old.print_info()


Node Depth: 1
Is leaf? False
Splits on: 'term_ 36 months'
Classification Error: 0.4963464431549538
Target value counts: [18748 18476]

Node Depth: 2
Is leaf? False
Splits on: 'grade_A'
Classification Error: 0.34923560663558495
Target value counts: [6002 3221]

Node Depth: 3
Is leaf? False
Splits on: 'grade_B'
Classification Error: 0.34630563472922604
Target value counts: [5963 3159]

Node Depth: 4
Is leaf? False
Splits on: 'grade_C'
Classification Error: 0.33415902898191724
Target value counts: [5376 2698]

Node Depth: 5
Is leaf? False
Splits on: 'grade_D'
Classification Error: 0.30319510537049627
Target value counts: [4100 1784]

Node Depth: 6
Is leaf? True
Splits on: 'N/A'
Classification Error: 0.2773131207527444
Target value counts: [2765 1061]

Node Depth: 6
Is leaf? True
Splits on: 'N/A'
Classification Error: 0.35131195335276966
Target value counts: [1335  723]

Node Depth: 5
Is leaf? False
Splits on: 'grade_D'
Classification Error: 0.41735159817351597
Target value counts: [1276 

# Quiz question
For my_decision_tree_new trained with max_depth = 6, min_node_size = 100, min_error_reduction=0.0, is the prediction path for validation_set[0] shorter, longer, or the same as for my_decision_tree_old that ignored the early stopping conditions 2 and 3?

In [129]:
dt_new.classify(df_valid.iloc[0:1], annotate=True)

Split 1 rows on term_ 36 months = 0
Split 1 rows on grade_A = 0
At leaf, 1 rows predicted 0


24    0
dtype: int64

In [130]:
dt_old.classify(df_valid.iloc[0:1], annotate=True)

Split 1 rows on term_ 36 months = 0
Split 1 rows on grade_A = 0
Split 1 rows on grade_B = 0
Split 1 rows on grade_C = 0
Split 1 rows on grade_D = 1
At leaf, 1 rows predicted 0


24    0
dtype: int64

In [131]:
print("Much shorter")

Much shorter


# Quiz question
For my_decision_tree_new trained with max_depth = 6, min_node_size = 100, min_error_reduction=0.0, is the prediction path for any point always shorter, always longer, always the same, shorter or the same, or longer or the same as for my_decision_tree_old that ignored the early stopping conditions 2 and 3?

In [132]:
print("shorter or the same")

shorter or the same


# Quiz question
For a tree trained on any dataset using max_depth = 6, min_node_size = 100, min_error_reduction=0.0, what is the maximum number of splits encountered while making a single prediction?

In [133]:
print("Max possible splits is max_depth-1 or 5 in this case")

Max possible splits is max_depth-1 or 5 in this case


# Quiz question
Is the validation error of the new decision tree (using early stopping conditions 2 and 3) lower than, higher than, or the same as that of the old decision tree from the previous assignment?

In [134]:
print("Error of new tree = {0}".format(1 - dt_new.score(df_valid, df_valid[target])))
print("Error of old tree = {0}".format(1 - dt_old.score(df_valid, df_valid[target])))
print("Error of new tree is slightly lower on validation set")

Error of new tree = 0.3832399827660491
Error of old tree = 0.3832399827660491
Error of new tree is slightly lower on validation set


In [162]:
dt1 = DecisionTree(df_train, target, max_depth=2, min_node_size=0, min_error_reduction=-1)
dt2 = DecisionTree(df_train, target, max_depth=6, min_node_size=0, min_error_reduction=-1)
dt3 = DecisionTree(df_train, target, max_depth=14, min_node_size=0, min_error_reduction=-1)

In [164]:
models = [dt1, dt2, dt3]
index_names = ['Model 1 (depth=2)', 'Model 2 (depth=6)', 'Model 3 (depth=14)']
df_errors = pd.concat([pd.Series([1- t.score(df_train, df_train[target]) for t in models], name='Training', index=index_names), pd.Series([1 - t.score(df_valid, df_valid[target]) for t in models], name='Validation', index=index_names)], axis=1)

# Quiz Question
Which tree has the smallest error on the validation data?

# Quiz Question
Does the tree with the smallest error in the training data also have the smallest error in the validation data?

# Quiz Question
Is it always true that the tree with the lowest classification error on the training set will result in the lowest classification error in the validation set?

In [165]:
display(df_errors)

,Training,Validation
Model 1 (depth=2),0.421637,0.423309
Model 2 (depth=6),0.385907,0.383240
Model 3 (depth=14),0.377391,0.458531


# Quiz question
Which tree has the largest complexity?

# Quiz question
Is it always true that the most complex tree will result in the lowest classification error in the validation_set?

In [166]:
pd.Series([t.count_leaves() for t in models], name='Model Complexity (Leaf Count)', index=index_names)

Model 1 (depth=2)      2
Model 2 (depth=6)     14
Model 3 (depth=14)    90
Name: Model Complexity (Leaf Count), dtype: int64

In [167]:
error_reductions = [-1., 0., .05]
index_names = ['Model 4 (ER=-1)', 'Model 5 (ER=0)', 'Model 6 (ER=0.05)']
models2 = [DecisionTree(df_train, target, max_depth=6, min_node_size=0, min_error_reduction=e) for e in error_reductions]

In [168]:
df_models2 = pd.concat([pd.Series([1-t.score(df_valid, df_valid[target]) for t in models2], name='Validation Error', index=index_names), pd.Series([t.count_leaves() for t in models2], name='Complexity (Leaf Count)', index=index_names)], axis=1)

# Quiz Question
Using the complexity definition above, which model (model_4, model_5, or model_6) has the largest complexity? Did this match your expectation?

# Quiz Question
model_4 and model_5 have similar classification error on the validation set but model_5 has lower complexity? Should you pick model_5 over model_4?

In [169]:
df_models2

,Validation Error,Complexity (Leaf Count)
Model 4 (ER=-1),0.383240,14
Model 5 (ER=0),0.383240,9
Model 6 (ER=0.05),0.423309,2


In [170]:
min_node_sizes = [0, 2000, 50000]
index_names = ['Model 7 (MNS=0)', 'Model 8 (MNS=2000)', 'Model 9 (MNS=50000)']
models3 = [DecisionTree(df_train, target, max_depth=6, min_node_size=n, min_error_reduction=-1) for n in min_node_sizes]

In [171]:
df_models3 = pd.concat([pd.Series([1-t.score(df_valid, df_valid[target]) for t in models3], name='Validation Error', index=index_names), pd.Series([t.count_leaves() for t in models3], name='Complexity (Leaf Count)', index=index_names)], axis=1)

# Quiz Question
Using the results obtained in this section, which model (model_7, model_8, or model_9) would you choose to use?

In [172]:
df_models3

,Validation Error,Complexity (Leaf Count)
Model 7 (MNS=0),0.383240,14
Model 8 (MNS=2000),0.382917,10
Model 9 (MNS=50000),0.503447,1
